## Thoughts

- smoke of deceit for team could be really great feature
- find median of kills, last hits for team, not for every hero

In [ ]:
%matplotlib inline

from matplotlib import pyplot as plt
import pandas
import numpy as np

data = pandas.read_csv('./features.csv', index_col='match_id')

In [ ]:
def filter_columns(data):
    cols_to_remove = ['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire',
                      'barracks_status_radiant', 'barracks_status_dire']
    
    return data.drop(cols_to_remove, axis=1, inplace=False)

In [ ]:
def get_cols_with_missing_values(A):
    n = A.shape[0]
    vals = A.count().values
    axes = A.count().axes[0]
    
    col_indexes = [i for i, v in enumerate(vals) if v != n]
    res = [axes[i] for i in col_indexes]
    return res

In [ ]:
data_gb = filter_columns(data)
cols_with_missing_values = get_cols_with_missing_values(data_gb)
print 'Following columns are missing some values:'
print ', '.join(cols_with_missing_values)

In [ ]:
from datetime import datetime

# A decorator for measuring function execution time
def timeit(f):
    
    def timed(*args, **kwargs):
        start = datetime.now()
        res = f(*args, **kwargs)
        delta = datetime.now() - start
        print '{f} exec time: {t}'.format(f=f, t=delta)
        return result
        
    return timed

________
# Approach 1: gradient boosting


In [ ]:
# Preparing data

data_gb.fillna(0, inplace=True)
X = data_gb.values
y = data['radiant_win']

In [ ]:
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV

fold = KFold(X.shape[0], n_folds=2, shuffle=True, random_state=1337)
gbc = GradientBoostingClassifier(random_state=1337)

grid = {'n_estimators': [350]}
clf = GridSearchCV(estimator=gbc, param_grid=grid, scoring='accuracy', cv=fold, n_jobs=4, verbose=2)
clf.fit(X, y)

In [ ]:
scores = clf.grid_scores_
vals = [score.mean_validation_score for score in scores]
estimators = [score.parameters['n_estimators'] for score in scores]

f = plt.figure()
plt.plot(estimators, vals, 'bo-')

print vals
print estimators